## Set 1

This is the story of my journey through [the cryptopals crypto challenges](https://cryptopals.com/), a journey which has opened my eyes to the joys of looking at a string of letters and numbers and seeing a bunch of ones and zeroes. 

My background in crypto is minimal: I took a cryptography class in college, pass-fail, and I understood approximately none of it. Also, I've tried [converting strings from base64](https://tracking-game.reaktor.com/signal/vs/noise/play) and pretty much always ending up in an error state.

###  Challenge 1: Convert hex to base64

> The string:
>
> `49276d206b696c6c696e6720796f757220627261696e206c696b65206120706f69736f6e6f7573206d757368726f6f6d`
>
> Should produce
>
> `SSdtIGtpbGxpbmcgeW91ciBicmFpbiBsaWtlIGEgcG9pc29ub3VzIG11c2hyb29t`
>
> So go ahead and make that happen.

There's a little hint at the bottom of the challenge: "Always operate on raw bytes, never on encoded strings. Only use hex and base64 for pretty-printing."

---

A major theme through this experience is discovering the massive amounts of ambiguity living in the black boxes that make up any language more complicated than assembly code. Example: the _base-16 number_ `49276d206b...`, when converted to base64, should blah blah blah...

#### the beginning: ascii to hex

Let's say I have a string — which, for now, is defined as the result of concatenating zero or more ASCII characters. Here's an example string that I plucked out of nowhere: `I'm killing your brain like a poisonous mushroom`.

Each character in the string is represented internally with a number greater than or equal to zero and less than 2<sup>8</sup>. Let's represent them in base-16, because we can write any hexadecimal number less than 256 (i.e., 2<sup>8</sup>) with only two hex-digits.


| char | ASCII |
| :--- | ----- |
| I    | 49<sub>16</sub> |
| '    | 27<sub>16</sub> |
| m    | 6d<sub>16</sub> |
| \[Space\] | 20<sub>16</sub> |
| k    | 6b<sub>16</sub> |


and so on. If you concatenate all those ASCII values from left to right, you'll get our mislabelled _base-16 number_ `49276d206b...`. I added whitespace to make it more readable.

#### hex to "raw bytes"

A base-16 number written with two hex-digits can be translated to a base-2 number written with eight bits (possibly including some leading zeroes).

| char | hex             | binary
| :--- | -----           | ------
| I    | 49<sub>16</sub> | 01001001<sub>2</sub>
| '    | 27<sub>16</sub> | 00100111<sub>2</sub>
| m    | 6d<sub>16</sub> | 01101101<sub>2</sub>
| \[Space\] | 20<sub>16</sub> | 00100000<sub>2</sub>
| k    | 6b<sub>16</sub> | 01101011<sub>2</sub>

and so on, again. If we concatenate the first three of those, we get 24 bits, which I've divided with whitespace into chunks of eight.

    01001001 00100111 01101101

#### "raw bytes" to base64

Base64 takes three bytes — that is, three groups of eight bits each — and re-imagines them as four groups of six bits each:

    01001001 00100111 01101101 =>
    010010 010010 011101 101101
    
The last step is looking up each six-bit chunk in [a table](https://en.wikipedia.org/wiki/Base64#Base64_table) and translating it to a letter, number, or math symbol as appropriate.

    010010 010010 011101 101101 => SSdt
    
Problem disambiguated!

### [Challenge 1](https://cryptopals.com/sets/1/challenges/1) 

There's a little hint at the bottom of the challenge: "Always operate on raw bytes, never on encoded strings. Only use hex and base64 for pretty-printing."

Strings in Python are confusing. I mean, not normally. Normally they encapsulate all the blips and bloops happening underneath and at worst you have to think of them as a list of letters. But today we're drilling down till we find the pure natural ore of zeroes and ones that strings are built upon, and we're going to have to power through some slag to get there.

The problem reads as follows:

> The string:

    49276d206b696c6c696e6720796f757220627261696e206c696b65206120706f69736f6e6f7573206d757368726f6f6d
    
> Should produce

    SSdtIGtpbGxpbmcgeW91ciBicmFpbiBsaWtlIGEgcG9pc29ub3VzIG11c2hyb29t
    
> So go ahead and make that happen.

Which is not quite right. Because while `49276d206b696c6c696e6720796f757220627261696e206c696b65206120706f69736f6e6f7573206d757368726f6f6d` is a string, you can plug it into the `bytes.fromhex` method and get something that looks like an even better string.

In [1]:
bytes.fromhex('49276d206b696c6c696e6720796f757220627261696e206c696b65206120706f69736f6e6f7573206d757368726f6f6d')

That `bytes` object (note that I haven't called anything a `str` yet) is what we want to encode. 49276d206b696c6c696e6720796f757220627261696e206c696b65206120706f69736f6e6f7573206d757368726f6f6d is a hexadecimal number, specifically the number that you get when you map each character in "I'm killing your brain like a poisonous mushroom" to its ASCII code, convert each base-10 ASCII code to a two character hexadecimal number (possibly including a leading zero) and concatenate all of those hexadecimal numbers into a single big number.

I'm going to make up some terminology just to clarify what's what.

In my world, there's __plaintext__. Plaintext is what we're encoding, and it's always of good old type `str`.

In [2]:
hello = 'hello world'

There's a __bytes__ object. It's [kinda well-documented](https://devdocs.io/python~3.7/library/stdtypes#bytes) in the Python docs.

In [3]:
by = bytes(hello, 'ascii')

In [4]:
by

There's also a __hex-encoded string__. A hex-encoded string is also of type `str` and all of its characters are in `string.hexdigits`. It's what you get when you run the algorithm I described above.

~~~py
hx = ''.join(hex(ord(i))[2:] for i in hello
~~~

I couldn't find a straightforward way to do this, but it looks to me like what's below isn't too bad.

In [5]:
hx = bytes.hex(by)

In [6]:
hx

There's a __hex-prefixed string__. This is what you get back from the `hex` built-in function. It starts with "0x", which makes it pretty useless.

In [7]:
n = int(hx, base=16)

In [8]:
n

126207244316550804821666916

In [9]:
hex(n)

'0x68656c6c6f20776f726c64'

With all of our different ways to represent the same concatenation of characters finally sorted out, the problem becomes solvable.

In [9]:
import base64

def to_base64(hexstring):
    return base64.b64encode(bytes.fromhex(hexstring))
    
to_base64('49276d206b696c6c696e6720796f757220627261696e206c696b65206120706f69736f6e6f7573206d757368726f6f6d')

b'SSdtIGtpbGxpbmcgeW91ciBicmFpbiBsaWtlIGEgcG9pc29ub3VzIG11c2hyb29t'

### [Challenge 2](https://cryptopals.com/sets/1/challenges/2)

Once again, you're given numbers as hex-encoded strings and asked to XOR them. Python has a bitwise XOR operator that takes two numbers as operands, so you could get away with just a little conversion:

In [11]:
m = '1c0111001f010100061a024b53535009181c'
n = '686974207468652062756c6c277320657965'

hex(int(m, base=16) ^ int(n, base=16))

'0x746865206b696420646f6e277420706c6179'

But looking ahead, I don't think that's exactly what they want. I suspect that this is what they're getting at, iterating over each pair of bytes one at a time, XORing them and converting the result:

In [66]:
mb = bytes.fromhex(m)
nb = bytes.fromhex(n)

bytes(x ^ y for x, y in zip(mb, nb)).hex()

'746865206b696420646f6e277420706c6179'

### [Challenge 3](https://cryptopals.com/sets/1/challenges/3)

They don't really explain what single-byte XOR means, so I'll do it here.

In [67]:
def single_byte_xor(encrypted, key):
    """
    Decrypt a message encoded with a single-byte key.
    
    :param encrypted: the message, a hex-encoded string
    :param key: the key, 0 <= key < 2 ** 8
    :return: the plaintext bytes
    """
    return bytes(c ^ key for c in bytes.fromhex(encrypted))

Taking a cue from above, or looking ahead to the next exercise, that function could also be written as:

~~~py
def single_byte_xor(encrypted, key):
    return bytes(x ^ y for x, y in zip(bytes.fromhex(encrypted), itertools.cycle(key)))
~~~

An assumption I'm making that isn't stated in the problem is that all the characters in the plaintext are _printable_.

TIL that _printable_ in Python means two different things. I assumed that for all strings `s` of length 1 where `s[0]` is an ASCII character, then `s.isprintable() == s in string.printable`. This is incorrect, and in particular, `'\n'.printable() != '\n' in string.printable`.

In [74]:
import string

def is_printable(text):
    return all(ch in string.printable for ch in text)

We'll take our given ciphertext, try each key from 1 (inclusive) to 2 ** 8 = 256 (exclusive), and see what we get.

In [101]:
def decrypt_single_byte_xor(encrypted):
    possible_plaintexts = [single_byte_xor(encrypted, key) for key in range(1, 2 ** 8)]
    
    # convert ASCII codes to characters
    possible_plaintexts = [list(map(lambda c: chr(c), p)) for p in possible_plaintexts]
    
    # assume all characters in plaintext are printable, as described above
    possible_plaintexts = list(filter(lambda t: is_printable(t), possible_plaintexts))
    
    # pretty-print
    return [''.join(t) for t in possible_plaintexts]

In [102]:
decrypt_single_byte_xor('1b37373331363f78151b7f2b783431333d78397828372d363c78373e783a393b3736')

['\\pptvqx?R\\8l?svtz?~?opjq{?py?}~|pq',
 'Q}}y{|u2_Q5a2~{yw2s2b}g|v2}t2psq}|',
 'Vzz~|{r5XV2f5y|~p5t5ez`{q5zs5wtvz{',
 'Txx|~yp7ZT0d7{~|r7v7gxbys7xq7uvtxy',
 'Kggcafo(EK/{(dacm(i(xg}fl(gn(jikgf',
 'Jffb`gn)DJ.z)e`bl)h)yf|gm)fo)khjfg',
 'Hdd`bel+FH,x+gb`n+j+{d~eo+dm+ijhde',
 'Nbbfdcj-@N*~-adfh-l-}bxci-bk-olnbc',
 'Maaeg`i.CM)}.bgek.o.~a{`j.ah.loma`',
 "Cooking MC's like a pound of bacon",
 'Bnnjhof!LB&r!mhjd!`!qntoe!ng!c`bno',
 'Ammikle"OA%q"nkig"c"rmwlf"md"`caml',
 '@llhjmd#N@$p#ojhf#b#slvmg#le#ab`lm',
 'Gkkomjc$IG#w$hmoa$e$tkqj`$kb$fegkj',
 'Fjjnlkb%HF"v%iln`%d%ujpka%jc%gdfjk',
 'Eiimoha&KE!u&jomc&g&vishb&i`&dgeih',
 "Dhhlni`'JD t'knlb'f'whric'ha'efdhi",
 'iEEACDM\ngi\rY\nFCAO\nK\nZE_DN\nEL\nHKIED',
 'hDD@BEL\x0bfh\x0cX\x0bGB@N\x0bJ\x0b[D^EO\x0bDM\x0bIJHDE',
 'oCCGEBK\x0cao\x0b_\x0c@EGI\x0cM\x0c\\CYBH\x0cCJ\x0cNMOCB',
 'nBBFDCJ\r`n\n^\rADFH\rL\r]BXCI\rBK\rOLNBC']

Only 21 keys produce something that looks like text, and it's easy to scan the list and discover one with English (mostly) words separated by spaces. But I assume it'll be useful to go through the second part of the instructions: 

> Devise some method for "scoring" a piece of English plaintext. Character frequency is a good metric.

Did some Googling and found a table of letter frequencies in English:

In [80]:
letter_frequencies = {
    'a': 0.0834,
    'b': 0.0154,
    'c': 0.0273,
    'd': 0.0414,
    'e': 0.126,
    'f': 0.0203,
    'g': 0.0192,
    'h': 0.0611,
    'i': 0.0671,
    'j': 0.0023,
    'k': 0.0086,
    'l': 0.0424,
    'm': 0.0253,
    'n': 0.068,
    'o': 0.077,
    'p': 0.0166,
    'q': 0.0009,
    'r': 0.0568,
    's': 0.0611,
    't': 0.0937,
    'u': 0.0285,
    'v': 0.0106,
    'w': 0.0234,
    'x': 0.002,
    'y': 0.0204,
    'z': 0.0006
}

This might need some refinement because I'm not a statistician, but it seemed reasonable to me to verify the (likelihood of) correctness of our English plaintext by summing, for each letter, the difference between the expected number of times the letter should appear in our plaintext and the actual number of times the letter does appear. There might be subtleties to this that I'm missing, but it worked for this challenge and the next one.

In [105]:
from collections import namedtuple

Score = namedtuple('Score', ['value', 'plaintext'])

nil_score = Score(float('inf'), plaintext=None)

#### Digression

I want to take a moment and share my love for the venerable `namedtuple`. It has everything you want in a data structure: metaprogrammatically-generated accessor methods, immutability, and... I guess that's it, but it's still amazing.

For my work-work, I write code in Java and Javascript, and the bane of my day is the constant scaffolding and re-scaffolding of these blobby containers across a bunch of different packages.

~~~java
package cryptopals.challenge.model

class Score {

    private final double score;
    private final String plaintext;
  
    private Score(Builder builder) {
        this.score = builder.score;
        this.plaintext = builder.plaintext;
    }
    
    public double getScore() { return score; }
    
    public String getPlaintext() { return plaintext; }
    
    public static Builder builder() { return new Builder(); }
    
    public static class Builder {
    
        private double score;
        private String plaintext;
        
        public void setScore(double score) {
            this.score = score;
            return this;
        }
        
        public void setPlaintext(String plaintext) {
            this.plaintext = plaintext;
            return this;
        }
        
        public Score build() {
            return new Score(this);
        }
    }
}
~~~

... compared to a single freaking line in Python! Rant over!

In [82]:
from collections import Counter

def score(text):
    """
    "Scores" the likelihood that a piece of text is in English.
    
    :param text: the text in question, as a str
    :return: a value indicating how close `text` is to expected English text, in terms of letter frequency. 
        The lower the value, the more likely the text is English.
    """
    # ignore non-letter characters
    counter = Counter(c.lower() for c in text if c.isalpha())
    letter_count = sum(counter.values())
    
    if not letter_count:
        return float('inf')
    
    total_variance = 0.0
    for letter, frequency in letter_frequencies.items():
        total_variance += abs(counter[letter] / letter_count - frequency)
        
    return total_variance

Including a call to `score` in our `decrypt_single_byte_xor` function: 

In [103]:
def decrypt_single_byte_xor(encrypted):
    possible_plaintexts = [single_byte_xor(encrypted, key) for key in range(1, 2 ** 8)]
    possible_plaintexts = [list(map(lambda c: chr(c), p)) for p in possible_plaintexts]
    possible_plaintexts = list(filter(lambda t: is_printable(t), possible_plaintexts))
    
    if not possible_plaintexts:
        return nil_score
    
    return min(map(lambda p: Score(score(p), ''.join(p)), possible_plaintexts))

In [92]:
decrypt_single_byte_xor('1b37373331363f78151b7f2b783431333d78397828372d363c78373e783a393b3736')

Score(value=0.8083555555555556, plaintext="Cooking MC's like a pound of bacon")

### Challenge 4

I don't see a way to do this without decrypting each line with every possible key until we find some plaintext that looks like English.

In [114]:
Decryption = namedtuple('Decryption', ['score', 'ciphertext', 'line_number'])

with open('s01_c04.txt') as file:    
    print(min((Decryption(decrypt_single_byte_xor(line.strip()), line.strip(), idx) 
               for idx, line in enumerate(file))))

Decryption(score=Score(value=0.6482, plaintext='Now that the party is jumping\n'), ciphertext='7b5a4215415d544115415d5015455447414c155c46155f4058455c5b523f', line_number=170)


### Challenge 5

Repeating-key XOR is the generalized case of single-byte XOR.

Given a `key`, each byte of the plaintext at index i is XOR'd with `key[i % len(key)]` and the results converted to two-character hexadecimal numbers, which are concatenated together.

In [118]:
import itertools

def repeating_key_xor(plaintext, key):
    """
    Encrypt plaintext with the given key.
    """
    encrypted = [p ^ k for p, k in zip((ord(c) for c in plaintext), itertools.cycle([ord(c) for c in key]))]
    return bytes(encrypted).hex()

In [119]:
repeating_key_xor("Burning 'em, if you ain't quick and nimble\\nI go crazy when I hear a cymbal", "ICE")

'0b3637272a2b2e63622c2e69692a23693a2a3c6324202d623d63343c2a26226324272765272a282b2f20152d0c69242a69203728393c69342d2c2d6500632d2c22376922652a3a282b2229'